In [1]:
import time
import random
from typing import List
import string
from transformers import AutoTokenizer
import transformers
import torch
import schema
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f'using device {device} for inference')

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device=device,
)


/home/chan/miniconda3/envs/langchain/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


using device cuda:0 for inference


Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.19s/it]


In [24]:

def gen_random_blog_text():
    prompt = "Please randomly write a blog\n"

    sequences = pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=100,
    )

    res: str = sequences[0]['generated_text']
    # res = res.strip(prompt)
    res.replace(prompt, "")

    return res

def gen_comment(blog: str):
    prompt = f'Please write a comment based on the content: \n {blog}'

    sequences = pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=100,
    )
    res: str = sequences[0]['generated_text']
    res = res.replace(prompt, "")

    return res

def get_random_string(length):
    # choose from all lowercase letter
    letters = string.ascii_lowercase
    result_str = ''.join(random.choice(letters) for i in range(length))

    return result_str


In [9]:
gen_random_blog_text()

'Please randomly write a blog\n\nPost about the following topic:\n\nThe Benefits of Meditation for Teenagers\n\nYou can write about how meditation can help teenagers to reduce stress and anxiety, improve their focus and concentration, and enhance their overall well-being. You can also include some tips on how to get started with meditation and some common misconceptions about meditation.\n\nRemember to keep your blog post informative,'

In [10]:
prompt = "Please randomly write a blog\n"
blog = gen_random_blog_text()

In [11]:
blog

'Please randomly write a blog\n\n  post on a topic of your choice.\n\n  Sure, here is a blog post on the topic of "The Importance of Taking Breaks at Work":\n\n  Title: The Importance of Taking Breaks at Work\n\n  As humans, we spend a significant portion of our lives working. Whether it\'s a 9-to-5 job, freelancing, or running our own businesses, work is an'

In [17]:
blog = blog.replace(prompt, "")
blog

'\n  post on a topic of your choice.\n\n  Sure, here is a blog post on the topic of "The Importance of Taking Breaks at Work":\n\n  Title: The Importance of Taking Breaks at Work\n\n  As humans, we spend a significant portion of our lives working. Whether it\'s a 9-to-5 job, freelancing, or running our own businesses, work is an'

In [31]:
comment = gen_comment(blog=blog)
comment

Input length of input_ids is 106, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


' integral'